In [332]:
import pandas as pd
import numpy as np

#需要优化的量

#关系矩阵
#dts
#M1=np.zeros((19,106),dtype=int) #进口：对应1
#M3=np.zeros((19,106),dtype=int) #国产：对应2
#ctd
#M2=np.zeros((1,18),dtype=int) #进口
#M4=np.zeros((1,18),dtype=int) #国产

#仓库状态矩阵
#O=np.zeros((20,1),dtype=int)  #是否开仓
#Z=np.zeros((20,1),dtype=int)  #是否智能


In [333]:
#参数导入
#国产，对应2
Q_dm=pd.read_csv('data_p/Q_dm.csv')
Q_dm=Q_dm.to_numpy()
Q_dm=Q_dm[:,1]
Q_dm=Q_dm.reshape((1,106))
Q_dm.shape

(1, 106)

In [334]:
#港口，对应1
Q_im=pd.read_csv('data_p/Q_im.csv')
Q_im=Q_im.to_numpy()
Q_im=Q_im[:,1]
Q_im=Q_im.reshape((1,106))
Q_dm.shape

(1, 106)

In [335]:
L_dts=pd.read_csv('data_p/d_matrix_dts.csv')
L_dts=L_dts.to_numpy()
L_dts=L_dts[1:]
L_dts=L_dts[:,2:]
L_dts_1=np.delete(L_dts,11,axis=0)  #im(delete neilu)
#k = 10 
#row_to_move = L_dts_1[k]
#L_dts_1 = np.delete(L_dts_1, k, axis=0)
#L_dts_1 = np.vstack([row_to_move, L_dts_1])

L_dts_2=np.delete(L_dts,6,axis=0)  #dm(delete gangkou)
#k = 6 
#row_to_move = L_dts_2[k]
#L_dts_2 = np.delete(L_dts_2, k, axis=0)
#L_dts_2 = np.vstack([row_to_move, L_dts_2])
L_dts_2.shape

(19, 106)

In [336]:
T_dts=pd.read_csv('data_p/t_matrix_dts.csv')
T_dts=T_dts.to_numpy()
T_dts=T_dts[1:]
T_dts=T_dts[:,2:]
T_dts_1=np.delete(T_dts,11,axis=0)#im(delete neilu)
T_dts_2=np.delete(T_dts,6,axis=0)#dm(delete gangkou)
T_dts_1.shape

(19, 106)

In [337]:
L_ctd=pd.read_csv('data_p/d_matrix_ctd.csv')
L_ctd=L_ctd.to_numpy()
L_ctd=L_ctd[:,2:]

In [338]:
T_ctd=pd.read_csv('data_p/t_matrix_ctd.csv')
T_ctd=T_ctd.to_numpy()
T_ctd=T_ctd[:,2:]

In [339]:
Pf=pd.read_csv('data_p/Pf.csv')
Pf=Pf.to_numpy()
Pf=Pf[:,1]
Pf.shape

(20,)

In [340]:
Of=pd.read_csv('data_p/Of.csv')
Of=Of.to_numpy()
Of=Of[:,1]
#Of.reshape((1,20))
Of.shape

(20,)

In [341]:
Cap=pd.read_csv('data_p/Cap.csv')
Cap=Cap.to_numpy()
Cap=Cap[:,1]
#Cap.reshape((1,20))
Cap.shape

(20,)

In [342]:
#约束条件
##时间
Q_all=Q_im.sum()
def time_point(M_im,M_dm):
    Q_out=0
    tmp1=M_im*T_dts_1
    tmp2=M_dm*T_dts_2  #这里改到对应的时间矩阵
    #print(tmp1,"\n",tmp2)
    tmp1=tmp1.sum(axis=0)
    tmp2=tmp2.sum(axis=0)
    for i in range(0,106):
        if tmp1[i]>=10:
            Q_out+=Q_im[0,i]
        if tmp2[i]>=10:
            Q_out+=Q_dm[0,i]
    out_point=Q_out/Q_all
    return out_point

def time_lim(M_im,M_dm):
    if time_point(M_im,M_dm)<=0.05:
        return True
    return False

#最大订单量不超过最大承载量
def cap_lim(M_im,M_dm):
    tmp=M_im*Q_im
    tmp1=tmp.sum(axis=1)
    tmp2=M_dm*Q_dm
    tmp3=tmp2.sum(axis=1)
    """
    s1=0
    s2=0
    for j in range(0,20):
        if j==6:
            sum=tmp1[s1]
            s2-=1
        elif j==11:
            sum=tmp3[s2]
            s1-=1
        else:
            sum=tmp1[s1]+tmp3[s2]
        if sum>Cap[j]:
            return False
        s1+=1
        s2+=1
    """
    
    return True

#cdc必须开
def cdc_op(O):
    return O[6]==1 and O[11]==1

#每个门店对应一个商店
def sd_lim(M_1,M_3):
    M1_s=M_1.sum(axis=0)
    #print(M1_s.shape)
    M2_s=M_3.sum(axis=0)
    #print(M2_s)
    for i in M1_s:
        if i!=1:
            return False
    for i in M2_s:
        if i!=1:
            return False
    return True

#每个rdc对应cdc
def rdc_cdc(k,M_1,M_2):
    M_1_rdc = np.delete(M_1,k)
    for i in range(18):
        if M_1_rdc[i] != 0:
            M_2[0][i] = 1
        else:
            M_2[0][i] = 0

In [343]:
#目标优化式
##处置成本
def ccl_cp(Z,Q,M_s,M_d,k,Of,Pf):
    cp = 0
    Q_dc = M_s * Q 
    Q_dc = Q_dc.sum(axis=1) # dc(rdc+一个cdc)的订单量
    #print(Q_c) #19,一维数组
    #tmp1=tmp1.reshape((1,19))
    Q_rdc = np.delete(Q_dc,k) # rdc的订单量
    Pf = np.delete(Pf,k) 
    Of = np.delete(Of,k)
    Z = np.delete(Z,k)
    tmp = M_d * Q_rdc
    ccp = tmp.sum()
    m = -0.5*Z+1
    for i in range(0,19):
        cp += Of[i] + Q_dc[i] * Pf[i] * m[i] + Z[i]
    print(cp)
    return cp+ccp

#运输成本
def ccl_ct(k,Q,M_s,M_d,L_s,L_d):
    sum=0
    tmp1 = M_s * Q * L_s * 1.25
    Q_c = Q @ M_s.T
    Q_c = np.delete(Q_c,k,axis=1) 
    tmp2 = M_d * Q_c * L_d.T * 0.6
    sum+=tmp1.sum()+tmp2.sum()
    return sum

#总成本
def cal_all(M1,M2,M3,M4,O,Z,Of,Pf,L_ctd,L_dts_1,L_dts_2):
    M1=M1.tolist()
    M1=np.array(M1)
    M2=M2.tolist()
    M2=np.array(M2)
    M3=M3.tolist()
    M3=np.array(M3)
    M4=M4.tolist()
    M4=np.array(M4)
    O=O.tolist()
    O=np.array(O)
    Z=Z.tolist()
    #print(Z[0])
    Z=np.array(Z)
    c=0
    #进口
    c+=ccl_cp(Z,Q_im,M1,M2,6,Of,Pf)
    #print(c)
    c+=ccl_ct(10,Q_im,M1,M2,L_dts_1,L_ctd[0])
    #print(c)
    #国产
    c+=ccl_cp(Z,Q_dm,M3,M4,10,Of,Pf)
    #print(c)
    c+=ccl_ct(6,Q_dm,M3,M4,L_dts_2,L_ctd[1])
    #print(c)
    return c

In [344]:
from gurobipy import Model, GRB, quicksum
import numpy as np

# 初始化模型
model = Model('Optimization')

# 定义决策变量
M1 = model.addMVar(shape=(19, 106), vtype=GRB.BINARY, name="M1")
M2 = model.addMVar(shape=(1, 18), vtype=GRB.BINARY, name="M2")
M3 = model.addMVar(shape=(19, 106), vtype=GRB.BINARY, name="M3")
M4 = model.addMVar(shape=(1, 18), vtype=GRB.BINARY, name="M4")
#O = model.addMVar(shape=(20, 1), vtype=GRB.BINARY, name="O")
Z = model.addMVar(shape=(20, 1), vtype=GRB.BINARY, name="Z")

# 定义约束条件

for j in range(20):
    if j ==6 or j == 10:
        continue
    elif j < 6:
        model.addConstr(quicksum(M1[j, i] * Q_im[0, i] + M3[j, i] * Q_dm[0, i] for i in range(106)) <= Cap[j], "Cap_Limit_{}".format(j))
    elif 6 < j < 10:
        model.addConstr(quicksum(M1[j, i] * Q_im[0, i] + M3[j-1, i] * Q_dm[0, i] for i in range(106)) <= Cap[j], "Cap_Limit_{}".format(j))
    else:
        model.addConstr(quicksum(M1[j-1, i] * Q_im[0, i] + M3[j-1, i] * Q_dm[0, i] for i in range(106)) <= Cap[j], "Cap_Limit_{}".format(j))


# cdc 必须开
#model.addConstr(O[6] == 1, "CDC_Open_6")
#model.addConstr(O[11] == 1, "CDC_Open_11")

# 每个门店对应一个仓库
model.addConstrs((quicksum(M1[:, i]) == 1 for i in range(106)), "Store_1")
model.addConstrs((quicksum(M3[:, i]) == 1 for i in range(106)), "Store_3")

def rdc_cdc_constraint(model, k, M_1, M_2, M):
    for i in range(18):
        # 去除第k行的约束表达式
        M_1_rdc = quicksum(M_1[j, i] for j in range(19) if j != k)
        
        # 定义二进制变量，用于条件判断
        binary_var = model.addVar(vtype=GRB.BINARY, name=f"binary_var_{i}")

        # 约束1: M_1_rdc > 0 implies M_2[0, i] = 1 (实现 M_2[0, i] >= binary_var)
        model.addConstr(M_2[0, i] >= binary_var, f"rdc_cdc_cond1_{i}")

        # 约束2: M_1_rdc <= 0 implies M_2[0, i] = 0 (实现 binary_var <= M_1_rdc / M)
        model.addConstr(binary_var * M >= M_1_rdc, f"rdc_cdc_cond2_{i}")

        # 约束3: 当M_1_rdc为0时，M_2[0, i]必须为0
        model.addConstr(binary_var <= M_2[0, i], f"rdc_cdc_cond3_{i}")

rdc_cdc_constraint(model, k=5, M_1=M1, M_2=M2, M=1000)
rdc_cdc_constraint(model, k=5, M_1=M3, M_2=M4, M=1000)

# 设置目标函数
model.setObjective(cal_all(M1,M2,M3,M4,O,Z,Of,Pf,L_ctd,L_dts_1,L_dts_2), GRB.MINIMIZE)

# 求解模型
model.optimize()

# 输出结果
if model.status == GRB.OPTIMAL:
    print('Optimal Objective Value:', model.ObjVal)
    
    # 将结果转为pandas DataFrame
    M1_df = pd.DataFrame(M1.X)
    M2_df = pd.DataFrame(M2.X)
    M3_df = pd.DataFrame(M3.X)
    M4_df = pd.DataFrame(M4.X)
    #O_df = pd.DataFrame(O.X)
    Z_df = pd.DataFrame(Z.X)
    
    # 创建一个Excel writer对象
    with pd.ExcelWriter('optimization_results.xlsx') as writer:
        M1_df.to_excel(writer, sheet_name='M1', index=False)
        M2_df.to_excel(writer, sheet_name='M2', index=False)
        M3_df.to_excel(writer, sheet_name='M3', index=False)
        M4_df.to_excel(writer, sheet_name='M4', index=False)
        #O_df.to_excel(writer, sheet_name='O', index=False)
        Z_df.to_excel(writer, sheet_name='Z', index=False)
        
    print("Results saved to 'optimization_results.xlsx'")
else:
    print('No optimal solution found.')

450.0 + 0.11169000000000001 <gurobi.Var *Awaiting Model Update*> + 0.17228000000000002 <gurobi.Var *Awaiting Model Update*> + 0.08103 <gurobi.Var *Awaiting Model Update*> + 2.3987800000000004 <gurobi.Var *Awaiting Model Update*> + 0.05621 <gurobi.Var *Awaiting Model Update*> + 0.11899000000000001 <gurobi.Var *Awaiting Model Update*> + 0.33433999999999997 <gurobi.Var *Awaiting Model Update*> + 0.35624 <gurobi.Var *Awaiting Model Update*> + 0.26206999999999997 <gurobi.Var *Awaiting Model Update*> + 0.1095 <gurobi.Var *Awaiting Model Update*> + 1.0585 <gurobi.Var *Awaiting Model Update*> + 0.66795 <gurobi.Var *Awaiting Model Update*> + 0.12483000000000001 <gurobi.Var *Awaiting Model Update*> + 0.36135 <gurobi.Var *Awaiting Model Update*> + 0.08395 <gurobi.Var *Awaiting Model Update*> + 0.057670000000000006 <gurobi.Var *Awaiting Model Update*> + 0.30513999999999997 <gurobi.Var *Awaiting Model Update*> + 0.07738 <gurobi.Var *Awaiting Model Update*> + 0.25988 <gurobi.Var *Awaiting Model Upda